<a href="https://colab.research.google.com/github/scspinney/pgm-fmri/blob/main/fmri_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You might need to upload the files into the collab directory. The files used in this script are located inside the example dataset folder : 
 

*   sub-01/func/sub-01_task-stopsignal_run-01_bold.nii.gz
*   sub-02/func/sub-02_task-stopsignal_run-01_bold.nii.gz


In [1]:
!pip install nilearn

     |████████████████████████████████| 3.0MB 5.9MB/s 


In [61]:
import numpy as np 
import os
from nilearn.image import index_img, smooth_img

The datasets we will be using will be a set of nifti files, so we need a library which can read an manipulate those files into matrices. Nilearn is standard for python, and it has the tools we need to handle this type of data (see example dataset in README). We can build a list of 4D files, then read them one-by-one and extract the slices of interest. 


In [62]:
sub1_path = '/sub-01_task-stopsignal_run-01_bold.nii.gz'
sub2_path = '/sub-02_task-stopsignal_run-01_bold.nii.gz'
#sub1_path = '~/Downloads/open-neuro/sub-01/func/sub-01_task-stopsignal_run-01_bold.nii.gz'
#sub2_path = '~/Downloads/open-neuro/sub-02/func/sub-02_task-stopsignal_run-01_bold.nii.gz'

images = [sub1_path,sub2_path]

In [63]:
# list of NiftiImage objects
X = []

for index, image_path in enumerate(images):
    # load image and remove nan and inf values.
    # applying smooth_img to an image with fwhm=None simply cleans up
    # non-finite values but otherwise doesn't modify the image.
    image = smooth_img(image_path, fwhm=None)
    X.append(image)

In [64]:
print(type(X[0]))
print(type(X[0].dataobj))
X[0].shape

<class 'nibabel.nifti1.Nifti1Image'>
<class 'numpy.ndarray'>


(64, 64, 30, 182)

Then you can grab the image into a numpy ndarray easily by calling dataobj on each [NiftiImage](https://nipy.org/nibabel/reference/nibabel.nifti1.html#nibabel.nifti1.Nifti1Image) object in the list X. The last dimension is time (the number of 3D slices taken during the course of the task):

In [65]:
numpy_4d_img = X[0].dataobj
numpy_4d_img.shape 

(64, 64, 30, 182)

You can work directly with the NIftiImage object to retrieve slices also (see Nilearn API; it may be less costly in memory to read just the slices we need, rather than loading the entire 4d array). This will be useful when we try to select slices (single 3D images from the time series) that are relevant to the task e.g. the N slices before the participant sees a rewarding cue on the screen is of interest if we want to model fMRI activation leading up to reward (reward anticipation).

In [66]:
slice_50 = index_img(X[0], 50) # grab slice number 50 into a 
slice_50_60 = index_img(X[0], slice(50, 60)) # grab slice number 50 to 60 

slice_50.shape, slice_50_60.shape

((64, 64, 30), (64, 64, 30, 10))